seq2seq French - Eng demo 

In [1]:
import pandas as pd 
lines = pd.read_csv('data/fra.txt', header= None, names = ['src', 'tar', 'att'], sep='\t')
len(lines)

177210

In [2]:
# take only 60K 
lines = lines[0:60000]
lines.sample(10)

,src,tar,att
14158,He has no hat on.,Il ne porte pas de chapeau.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
24389,I may die tomorrow.,Je mourrai peut-être demain.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
21512,Tom lives near us.,Tom habite près de chez nous.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
840,Let me in.,Laissez-moi rentrer.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
5227,He's adorable.,Il est adorable.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
29466,I drive a black car.,Je conduis une voiture noire.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
45723,Who helps your mother?,Qui aide votre mère ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
24787,I'll foot the bill.,Je paierai l'addition.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
18900,I bought the book.,J'ai acheté le livre.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
48105,I don't need an office.,Je n'ai pas besoin de bureau.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [3]:
lines = lines[['src','tar']] # taking off the att
lines.sample(10)

,src,tar
34772,He's a computer nerd.,C'est un informaticien boutonneux.
19054,I feel better now.,"Je me sens mieux, maintenant."
4704,Years passed.,Des années ont passé.
4550,Was it funny?,Était-ce drôle ?
15343,I'm an alcoholic.,Je suis alcoolique.
49269,I'm honored to be here.,Je suis honoré d'être ici.
13618,Are you on crack?,Êtes-vous droguée au crack ?
49189,I'm afraid of the dark.,J'ai peur du noir.
9687,Why can't I go?,Pourquoi est-ce que je ne peux pas y aller ?
21070,The soup is thick.,La soupe est épaisse.


In [4]:
# need to add <sos> for initiating and <eos> for ending 
lines.tar = lines.tar.apply(lambda x: '\t ' + x + ' \n')
lines.sample(10)

,src,tar
13216,Why did you lie?,\t Pourquoi as-tu menti ? \n
36655,I've found a new job.,\t J'ai trouvé un nouveau travail. \n
42324,I suppose that's true.,\t Je suppose que c'est vrai. \n
11302,I want to do it.,\t Je veux le faire. \n
27992,You're very clever.,\t Vous êtes très ingénieuses. \n
23872,I am not a teacher.,\t Je ne suis pas un enseignant. \n
55158,I know how busy you are.,\t Je sais combien tu es occupé. \n
44899,Tom is a pediatrician.,\t Tom est pédiatre. \n
48620,I promised not to tell.,\t Je promis de ne rien dire. \n
18441,Have you finished?,\t As-tu fini ? \n


In [5]:
# vocabs: 

src_vocab=set()
for line in lines.src: 
    for char in line: 
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [6]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

79
105


영어와 프랑스어는 각각 약 80개와 100개의 글자가 존재

In [7]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']


In [8]:
# applying index to char
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, ',': 11, '-': 12, '.': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 3

In [9]:
# encoder (eng): 
encoder_input = []
for line in lines.src: 
    temp_x =[]
    
    for w in line: 
        temp_x.append(src_to_index[w]) # convert w to int
        
    encoder_input.append(temp_x)

print(encoder_input[:5])

[[30, 64, 10], [31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2]]


In [10]:
# decoder (fra):
decoder_input =[]
for line in lines.tar:
    temp_x = []
    
    for w in line: 
        temp_x.append(tar_to_index[w])
    decoder_input.append(temp_x)

print(decoder_input[:5])

[[1, 3, 47, 52, 3, 4, 3, 2], [1, 3, 44, 52, 63, 72, 71, 3, 4, 3, 2], [1, 3, 44, 52, 63, 72, 71, 13, 3, 2], [1, 3, 28, 66, 72, 69, 70, 104, 4, 3, 2], [1, 3, 28, 66, 72, 69, 56, 77, 104, 4, 3, 2]]


디코더의 예측값과 비교하기 위한 실제값이 필요. 그런데 이 실제값에는 시작 심볼에 해당되는 sos 가 있을 필요가 없음! 즉, 처음 값 ('\t')들 제거

In [11]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
        if t>0:
            temp_X.append(tar_to_index[w])
        t+=1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 47, 52, 3, 4, 3, 2], [3, 44, 52, 63, 72, 71, 3, 4, 3, 2], [3, 44, 52, 63, 72, 71, 13, 3, 2], [3, 28, 66, 72, 69, 70, 104, 4, 3, 2], [3, 28, 66, 72, 69, 56, 77, 104, 4, 3, 2]]


In [12]:
# padding:

max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

25
76


병렬 데이터는 영어와 프랑스어의 길이는 하나의 쌍이라고 하더라도 전부 다르므로 패딩을 할 때도 이 두 개의 데이터의 길이를 전부 동일하게 맞춰줄 필요는 없음. 영어 데이터는 영어 샘플들끼리, 프랑스어는 프랑스어 샘플들끼리 길이를 맞추어서 패딩

In [13]:
from keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

Using TensorFlow backend.


In [14]:
# One hot encoding: 
from keras.utils import to_categorical

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는데 decoder_input이 왜 필요 이유: 

    * 훈련 과정에서는 이전 시점의 디코더 셀 출력값을 현 시점 디코더 셀의 입력값으로 넣지 않고, 이전 시점의 실제값을 현 시점의 디코더 셀 값으로 사용! 
        - 이유: 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 함 --> 훈련 시간 저하
        
이 방법의 이름이 Teacher Forcing!!

In [15]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)# to pass the encoder's status to decoder: return_state = True
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [16]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [17]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 291s 6ms/step - loss: 0.7715 - val_loss: 0.6948
Epoch 2/50
48000/48000 [==============================] - 284s 6ms/step - loss: 0.4820 - val_loss: 0.5507
Epoch 3/50
48000/48000 [==============================] - 373s 8ms/step - loss: 0.3989 - val_loss: 0.4873
Epoch 4/50
48000/48000 [==============================] - 367s 8ms/step - loss: 0.3532 - val_loss: 0.4441
Epoch 5/50
48000/48000 [==============================] - 321s 7ms/step - loss: 0.3235 - val_loss: 0.4198
Epoch 6/50
48000/48000 [==============================] - 282s 6ms/step - loss: 0.3021 - val_loss: 0.4012
Epoch 7/50
48000/48000 [==============================] - 280s 6ms/step - loss: 0.2856 - val_loss: 0.3897
Epoch 8/50
48000/48000 [==============================] - 296s 6ms/step - loss: 0.2725 - val_loss: 0.3799
Epoch 9/

applying seq2seq translator
Process:
1. get the sentence's hidden and cell states that i want to traslate from encoder
2. send status and sos to decoder
3. until reaches \n decoder do translation

In [18]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [19]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [20]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [21]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [22]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # sentence index
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('Input:', lines.src[seq_index])
    print('Correct senetence:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) 
    print('Translated sentence:', decoded_sentence[:len(decoded_sentence)-1]) 

-----------------------------------
Input: Run!
Correct senetence:  Cours ! 
Translated sentence:  Cours ! 
-----------------------------------
Input: I lied.
Correct senetence:  J'ai menti. 
Translated sentence:  J'ai appris l'argent. 
-----------------------------------
Input: Come in.
Correct senetence:  Entre. 
Translated sentence:  Entre. 
-----------------------------------
Input: Hurry up.
Correct senetence:  Magnez-vous ! 
Translated sentence:  Magne-toi ! 
-----------------------------------
Input: We walked.
Correct senetence:  Nous sommes allées à pied. 
Translated sentence:  Nous sommes amis. 
